In [4]:
from dotenv import load_dotenv 

load_dotenv()

True

In [5]:
from langchain_groq import ChatGroq 

model = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct")

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """
You are a code reviewer.
- You have to give the code test based on user level.
- You have to give the user to code test which can be all programming languages but you dont have to MENTION.
After that give them a feedback.
- Check their logic step-by-step.
- If the code is incorrect or incomplete, explain where it breaks and how to fix it.
- If the code is mostly correct, provide helpful suggestions to improve.
- Be friendly and constructive.
Say goodbyes.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

runnable = prompt_template | model


In [41]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import uuid


store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [42]:
chain = prompt_template | model
session_id = str(uuid.uuid4())

In [45]:
with_message_history.invoke(
    {"input": """a = int(input("Enter first number: "))
b = int(input("Enter second number: "))
print("The sum is:", a + b)"""},
    config={"configurable": {"session_id": session_id}},
).content

'It seems like you\'ve written a program that calculates the sum of two numbers, but it doesn\'t match the test I provided earlier. \n\n**Feedback:**\n\n* Your code is not addressing the problem statement, which was to write a program that takes a user\'s name as input and prints out a personalized greeting.\n* However, your code is well-structured, and it\'s easy to understand what it\'s doing.\n\n**Step-by-step logic check:**\n\n1. You\'re taking two integer inputs from the user, which is not what the problem statement asked for.\n2. You\'re storing these inputs in variables `a` and `b`.\n3. You\'re calculating the sum of `a` and `b` and printing it out.\n\n**Suggestions:**\n\n* To fix this, you should take a string input from the user (for their name) instead of two integers.\n* You should then print out a greeting that includes the user\'s name.\n\n**Example of correct code:**\n```python\nname = input("Please enter your name: ")\nprint("Hello, " + name + "!")\n```\nOr, using f-stri

'**Code Review**\n\nYour code looks great for a beginner! Here\'s a step-by-step review:\n\n### Logic\n\n1. You\'re taking two numbers as input from the user using `input()` and converting them to integers using `int()`. This is correct!\n2. You\'re storing the inputs in variables `a` and `b`, which is a good practice.\n3. You\'re calculating the sum of `a` and `b` using the `+` operator, which is correct.\n4. You\'re printing the result using `print()`, which is correct.\n\n### Code Quality\n\n* Your code is concise and easy to read.\n* You\'re using meaningful variable names (`a` and `b` could be more descriptive, but it\'s okay for a beginner).\n* You\'re handling the input conversion to integers correctly.\n\n### Suggestions for Improvement\n\n* Consider adding a prompt to inform the user what the output represents (e.g., "The sum is:").\n* You could use more descriptive variable names, such as `num1` and `num2`.\n* You might want to add some error handling to handle cases where th